### В рамках исследования были рассмотрены Clickhouse и Vertica.

### Запуск

In [ ]:
!docker-compose up -d

In [ ]:
!export PYTHONPATH="${PYTHONPATH}:${PWD}/../.."

#### Загрузка тестовых данных в Clickhouse

In [1]:
from multiprocessing import Pool

import tqdm as tqdm
from clickhouse_driver import Client

from utils.fake_data_gen import Row, generate_fake_data
from config import CLICKHOUSE_HOST, NUMBER_OF_BATCHES, UPLOAD_BATCH_SIZE

client = Client(CLICKHOUSE_HOST)


def upload_batch(batch):
    columns = ', '.join(Row._fields)
    client.execute(
        f'INSERT INTO views ({columns}) VALUES',
        batch
    )

In [2]:
test_data = generate_fake_data(UPLOAD_BATCH_SIZE, NUMBER_OF_BATCHES)

with Pool() as pool:
    r = list(tqdm.tqdm(
        pool.imap(upload_batch, test_data),
        total=NUMBER_OF_BATCHES
    ))

100%|██████████| 1000/1000 [02:10<00:00,  7.65it/s]


#### Загрузка тестовых данных в Vertica

In [3]:
from multiprocessing import Pool

import tqdm
import vertica_python

from utils.fake_data_gen import Row, generate_fake_data
from config import NUMBER_OF_BATCHES, UPLOAD_BATCH_SIZE, VERTICA_CONNECTION_PARAMS


def upload_batch(batch):
    with vertica_python.connect(**VERTICA_CONNECTION_PARAMS) as connection:
        columns = ', '.join(Row._fields)
        placeholders = ', '.join(['%s'] * len(Row._fields))
        cursor = connection.cursor()
        cursor.executemany(
            f'INSERT INTO views ({columns}) VALUES ({placeholders})',
            batch
        )

In [4]:
test_data = generate_fake_data(UPLOAD_BATCH_SIZE, NUMBER_OF_BATCHES)
with Pool() as pool:
    r = list(tqdm.tqdm(
        pool.imap(upload_batch, test_data),
        total=NUMBER_OF_BATCHES
    ))

<generator object generate_fake_data.<locals>.<genexpr> at 0x7f561c320580>


100%|██████████| 1000/1000 [05:31<00:00,  3.02it/s]


### Выполнение тестовых запросов

Использовались следующие тестовые запросы:

In [5]:
from utils.test_queries import QUERIES

for name, query in QUERIES.items():
    print(f'{name}')
    print(f'{query}')
    print("----------")

average_movies_per_user
select avg(movies_watched) from ( select count(movie_id) as movies_watched from views group by user_id    ) as movies_count;
----------
average_view_times
select avg(viewed_frame) from views;
----------
top_20_users_by_total_view_time
select user_id, sum(viewed_frame) as view_time from views group by user_id order by view_time desc limit 20;
----------
top_20_movies_by_view_time
select movie_id, max(viewed_frame) as view_time from views group by movie_id order by view_time desc limit 20;
----------
unique_movies_count
select count(distinct movie_id) from views;
----------
unique_users_count
select count(distinct user_id) from views;
----------


#### Тестирование Clickhouse

In [6]:
from clickhouse_driver import Client

from utils.test_queries import QUERIES
from utils.timer import timer
from config import BENCHMARK_ITERATIONS, CLICKHOUSE_HOST

client = Client(CLICKHOUSE_HOST)


@timer(BENCHMARK_ITERATIONS)
def execute_query(query: str):
    client.execute(query)

In [7]:
for name, query in QUERIES.items():
    print(f'{name}')
    execute_query(query)

average_movies_per_user
Average execution time (over 10 runs): 0.8351 seconds

average_view_times
Average execution time (over 10 runs): 0.0668 seconds

top_20_users_by_total_view_time
Average execution time (over 10 runs): 0.6475 seconds

top_20_movies_by_view_time
Average execution time (over 10 runs): 0.3404 seconds

unique_movies_count
Average execution time (over 10 runs): 0.1960 seconds

unique_users_count
Average execution time (over 10 runs): 0.3617 seconds



#### Тестирование Vertica

In [8]:
import vertica_python

from utils.test_queries import QUERIES
from utils.timer import timer
from config import BENCHMARK_ITERATIONS, VERTICA_CONNECTION_PARAMS


@timer(BENCHMARK_ITERATIONS)
def execute_query(query: str):
    with vertica_python.connect(**VERTICA_CONNECTION_PARAMS) as connection:
        cursor = connection.cursor()
        cursor.execute(query)

In [9]:
for name, query in QUERIES.items():
    print(f'{name}')
    execute_query(query)

average_movies_per_user
Average execution time (over 10 runs): 1.5935 seconds

average_view_times
Average execution time (over 10 runs): 0.1601 seconds

top_20_users_by_total_view_time
Average execution time (over 10 runs): 1.5920 seconds

top_20_movies_by_view_time
Average execution time (over 10 runs): 1.4446 seconds

unique_movies_count
Average execution time (over 10 runs): 1.3468 seconds

unique_users_count
Average execution time (over 10 runs): 1.1616 seconds



### Вывод

Clickhouse лучше во всех тестах! 
Выбираем сlickhouse.

### Остановка

In [10]:
!docker-compose down -v

Stopping click_vertica_clickhouse_1 ... 
Stopping click_vertica_vertica_1    ... 
ping click_vertica_vertica_1    ... doneRemoving click_vertica_clickhouse_1 ... 
Removing click_vertica_vertica_1    ... 
Removing network click_vertica_default32mdone
Removing volume click_vertica_ugc_bench_clickhouse_data
Removing volume click_vertica_ugc_bench_vertica_data
